In [43]:
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql import functions as F
from pyspark.sql import types as T

# read the df and return
def extract_data(spark:SparkSession, path:str) -> DataFrame:
    return spark.read.option("header",True).csv(path)

# some transformation and agregation
def transform_data(df:DataFrame) -> DataFrame:
    oputput = (

        df
        .groupBy("manufacturer_name")
        .agg(
            F.count("manufacturer_name").alias("count_ads"),
            F.round(F.avg("year_produced")).cast(T.IntegerType()).alias("year_produced"),
            F.min(F.col("price_usd")).alias("min_price"),
            F.max(F.col("price_usd")).alias("max_price"),

        )
        .orderBy(F.col("count_ads"))

    )
    return oputput
# saving the results
def save_data_par(df:DataFrame,output_type:str)-> None:
    df.coalesce(4).write.mode("overwrite").format(output_type).save("output_type")

def save_data(df:DataFrame) -> None:
    df.coalesce(4).write.mode("overwrite").format("json").save("output_json")

def main():
    
    spark = SparkSession.builder.appName("Second").getOrCreate() # creation of spark session
    path = "data/cars.csv" # path to data
    #output_type = "json" # file type to save
    df = extract_data(spark,path)
    output = transform_data(df)
    # parametrized function
    # save_data_par(output,output_type)
    # simple save data
    save_data(output)

    # SparkContext stop() method
    spark.sparkContext.stop()

    #spark.stop()
    
main()

Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: org.apache.spark.SparkException: Only one SparkContext should be running in this JVM (see SPARK-2243).The currently running SparkContext was created at:
org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:490)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
py4j.commands.ConstructorCommand

In [41]:
spark.stop()